# Section 5 - Evaluation
The following script presents the first use case from Section 5 – Evaluation in the manuscript.

In [11]:
from QCloud import *
import statistics
from collections import defaultdict
import math

def print_result(job_records, LAMBDA): 
    finished = defaultdict(int)
    unfinished = defaultdict(int)
    wait_time = defaultdict(list)
    never_assigned = 0
    fidelity = defaultdict(list)

    avg_fidelity = defaultdict(int)
    std_fidelity = defaultdict(int)
    avg_waittime = defaultdict(int)
    std_waittime = defaultdict(int)

    for events in job_records.values():
        if 'arrival' in events and 'devc_start' in events and 'devc_finish' in events:            
            devc_name = events['devc_name'] 
            finished[devc_name] += 1
            wait_time_diff = events['devc_finish']-events['devc_start']
            wait_time[devc_name].append(wait_time_diff)
        elif 'arrival' in events and 'devc_start' in events and 'devc_finish' not in events:
            devc_name = events['devc_name']
            unfinished[devc_name] += 1        
        elif 'arrival' in events and 'devc_start' not in events:        
            never_assigned += 1
        if 'fidelity' in events: 
            devc_name = events['devc_name']
            fidelity[devc_name].append(events['fidelity'])


    
    total_job_processed = 0
    with open(f"results/counted-jobs-lambda-{LAMBDA}.txt", "a") as file:
        file.write(f"LAMBDA: {LAMBDA}\n")

        for key in finished.keys(): 
            avg_fidelity[key] = statistics.mean(fidelity[key])
            std_fidelity[key] = statistics.stdev(fidelity[key])
            avg_waittime[key] = round(statistics.mean(wait_time[key]), 4)
            std_waittime[key] = round(statistics.stdev(wait_time[key]), 4)

            total_job_processed += finished[key]
            
            # print(f"device name: {key}, finished: {finished[key]}, avg_fidelity: {avg_fidelity[key]:.4f} +/- {std_fidelity[key]:.4f}, avg_waittime: {avg_waittime[key]:.4f} +/- {std_waittime[key]:.4f}")    
            file.write(f"device name: {key}, finished: {finished[key]}, avg_fidelity: {avg_fidelity[key]:.4f} +/- {std_fidelity[key]:.4f}, avg_waittime: {avg_waittime[key]:.4f} +/- {std_waittime[key]:.4f}\n")
        print(f"LAMBDA: {LAMBDA} | Total job processed: {total_job_processed} | Never Assigned: {never_assigned}", )    
        file.write(f"Total job processed: {total_job_processed} | Never Assigned: {never_assigned}\n")
    
    
    sorted_finished = {key: finished[key] for key in sorted(finished, key=lambda x: int(x))}
    sorted_avg_waittime = {key: avg_waittime[key] for key in sorted(avg_waittime, key=lambda x: int(x))}
    
    print("Finished:", list(sorted_finished.values()))
    print("wait_time:", list(sorted_avg_waittime.values()))
    
    return total_job_processed, never_assigned

In [50]:
# deploying jobs from JSON file. 
sim_start_time = time.time()
LAMBDAS = [0.5, 1, 2, 3, 4, 5, 6]

processed = defaultdict(list)
notprocessed = defaultdict(list)

for i in range(1): 
    for LAMBDA in LAMBDAS: 
        # Create quantum devices
        ibm_kawasaki = IBM_Kawasaki(env=None, name="1", printlog = False)
        ibm_kyiv = IBM_Kyiv(env=None, name="2", printlog = False)
        ibm_sherbrooke = IBM_Sherbrooke(env=None, name="3", printlog = False)
        ibm_quebec = IBM_Quebec(env=None, name="4", printlog = False)
        ibm_rensselaer = IBM_Rensselaer(env=None, name="5", printlog = False)
        ibm_brisbane = IBM_Brisbane(env=None, name="6", printlog = False)        
        ibm_brussels = IBM_Brussels(env=None, name="7", printlog = False)
        ibm_strasbourg = IBM_Strasbourg(env=None, name="8", printlog = False)        
        ibm_marrakesh = IBM_Marrakesh(env=None, name="9", printlog = False) 
        ibm_fez = IBM_Fez(env=None, name="10", printlog = False)
        ibm_torino = IBM_Torino(env=None, name="11", printlog = False)


        expovar_model = lambda: random.expovariate(lambd=LAMBDA)
        devices = [ibm_kawasaki, ibm_kyiv, ibm_sherbrooke, ibm_quebec, ibm_rensselaer, 
                  ibm_brisbane, ibm_brussels, ibm_strasbourg, ibm_marrakesh, ibm_fez, ibm_torino]

        start_time = time.time() # Start the timer

        # Initialize and run the simulation using jobs from a CSV file
        qcloudsimenv = QCloudSimEnv(
            devices=devices,
            broker_class=ParallelBroker,
            job_feed_method="generator",
            job_generation_model=expovar_model
        )
        qcloudsimenv.run(until=10080)

        end_time = time.time() # End the timer
        elapsed_time = end_time - start_time # Calculate elapsed time

        print(f"Elapsed time: {elapsed_time:.5f} seconds\n")
        # Access job records after the simulation
        job_records = qcloudsimenv.job_records_manager.get_job_records()

        # Open a text file for writing
        with open(f"results/raw_records-lambda-{LAMBDA}.txt", "a") as file:
            file.write(f"Elapsed time: {elapsed_time:.5f} seconds\n")
            for job_id, events in job_records.items():
                # Write each job record to the file               
                file.write(f"Job ID: {job_id}, Events: {events}\n")
        processedjobs, notprocessedjobs = print_result(job_records, LAMBDA)
        processed[LAMBDA] = processedjobs
        notprocessed[LAMBDA] = notprocessedjobs

sim_end_time = time.time() # End the timer
sim_elapsed_time = sim_end_time - sim_start_time # Calculate elapsed time

print(f"Elapsed time: {sim_elapsed_time:.5f} seconds")

Elapsed time: 13.00808 seconds

LAMBDA: 0.5 | Total job processed: 4692 | Never Assigned: 316
Finished: [363, 395, 381, 417, 387, 485, 460, 445, 481, 446, 432]
wait_time: [250.87, 230.7598, 233.6642, 221.1464, 228.7627, 35.9038, 28.4647, 28.4668, 33.8066, 33.0204, 30.0406]
Elapsed time: 14.73066 seconds

LAMBDA: 1 | Total job processed: 7286 | Never Assigned: 2615
Finished: [374, 381, 372, 413, 403, 863, 905, 921, 890, 888, 876]
wait_time: [244.1629, 238.1356, 248.8756, 219.2056, 227.9619, 36.2678, 29.272, 29.893, 33.0437, 32.7094, 30.3696]
Elapsed time: 18.25884 seconds

LAMBDA: 2 | Total job processed: 12788 | Never Assigned: 7165
Finished: [369, 388, 407, 417, 413, 1769, 1780, 1813, 1793, 1834, 1805]
wait_time: [247.4935, 238.9643, 228.4039, 217.6904, 224.7599, 36.441, 29.2246, 29.7483, 32.8644, 33.5405, 31.3561]
Elapsed time: 31.16918 seconds

LAMBDA: 3 | Total job processed: 18086 | Never Assigned: 12064
Finished: [378, 378, 394, 411, 432, 2379, 2708, 2811, 2807, 2688, 2700]
wait_

In [2]:
# reunning after replacing the files. 
sim_start_time = time.time()
LAMBDAS = [0.5, 1, 2, 3, 4, 5, 6]

processed = defaultdict(list)
notprocessed = defaultdict(list)

for i in range(1): 
    for LAMBDA in LAMBDAS: 
        # Create quantum devices
        ibm_kawasaki = IBM_Kawasaki(env=None, name="1", printlog = False)
        ibm_kyiv = IBM_Kyiv(env=None, name="2", printlog = False)
        ibm_sherbrooke = IBM_Sherbrooke(env=None, name="3", printlog = False)
        ibm_quebec = IBM_Quebec(env=None, name="4", printlog = False)
        ibm_rensselaer = IBM_Rensselaer(env=None, name="5", printlog = False)
        ibm_brisbane = IBM_Brisbane(env=None, name="6", printlog = False)        
        ibm_brussels = IBM_Brussels(env=None, name="7", printlog = False)
        ibm_strasbourg = IBM_Strasbourg(env=None, name="8", printlog = False)        
        ibm_marrakesh = IBM_Marrakesh(env=None, name="9", printlog = False) 
        ibm_fez = IBM_Fez(env=None, name="10", printlog = False)
        ibm_torino = IBM_Torino(env=None, name="11", printlog = False)


        expovar_model = lambda: random.expovariate(lambd=LAMBDA)
        devices = [ibm_kawasaki, ibm_kyiv, ibm_sherbrooke, ibm_quebec, ibm_rensselaer, 
                  ibm_brisbane, ibm_brussels, ibm_strasbourg, ibm_marrakesh, ibm_fez, ibm_torino]

        start_time = time.time() # Start the timer

        # Initialize and run the simulation using jobs from a CSV file
        qcloudsimenv = QCloudSimEnv(
            devices=devices,
            broker_class=ParallelBroker,
            job_feed_method="generator",
            job_generation_model=expovar_model
        )
        qcloudsimenv.run(until=10080)

        end_time = time.time() # End the timer
        elapsed_time = end_time - start_time # Calculate elapsed time

        print(f"Elapsed time: {elapsed_time:.5f} seconds\n")
        # Access job records after the simulation
        job_records = qcloudsimenv.job_records_manager.get_job_records()

        # Open a text file for writing
        with open(f"results/raw_records-lambda-{LAMBDA}.txt", "a") as file:
            file.write(f"Elapsed time: {elapsed_time:.5f} seconds\n")
            for job_id, events in job_records.items():
                # Write each job record to the file               
                file.write(f"Job ID: {job_id}, Events: {events}\n")
        processedjobs, notprocessedjobs = print_result(job_records, LAMBDA)
        processed[LAMBDA] = processedjobs
        notprocessed[LAMBDA] = notprocessedjobs

sim_end_time = time.time() # End the timer
sim_elapsed_time = sim_end_time - sim_start_time # Calculate elapsed time

print(f"Elapsed time: {sim_elapsed_time:.5f} seconds")

Elapsed time: 12.26485 seconds

LAMBDA: 0.5 | Total job processed: 4676 | Never Assigned: 300
Finished: [370, 374, 400, 401, 408, 437, 458, 491, 429, 433, 475]
wait_time: [250.5337, 239.7211, 224.2075, 214.6052, 220.9169, 36.8385, 29.1705, 27.8389, 32.3815, 32.3588, 31.3215]
Elapsed time: 15.63442 seconds

LAMBDA: 1 | Total job processed: 7398 | Never Assigned: 2534
Finished: [379, 392, 402, 409, 395, 887, 877, 873, 922, 921, 941]
wait_time: [247.9329, 229.0879, 234.9119, 223.6337, 227.3527, 35.5364, 29.6897, 29.2296, 32.556, 33.597, 30.852]
Elapsed time: 18.92147 seconds

LAMBDA: 2 | Total job processed: 12859 | Never Assigned: 7380
Finished: [406, 375, 390, 392, 412, 1885, 1792, 1847, 1792, 1731, 1837]
wait_time: [229.0045, 242.6137, 234.8515, 230.0333, 224.7162, 37.8611, 29.6567, 29.1787, 32.8945, 32.7679, 31.2026]
Elapsed time: 31.52828 seconds

LAMBDA: 3 | Total job processed: 18088 | Never Assigned: 12154
Finished: [389, 402, 398, 438, 405, 2368, 2744, 2743, 2690, 2752, 2759]
wai

In [13]:
# deploying jobs from JSON file. 
sim_start_time = time.time()
LAMBDAS = [0.5, 1, 2, 3, 4, 5, 6]

processed = defaultdict(list)
notprocessed = defaultdict(list)

for i in range(1): 
    for LAMBDA in LAMBDAS: 
        # Create quantum devices
        ibm_kawasaki = IBM_Kawasaki(env=None, name="1", printlog = False)
        ibm_kyiv = IBM_Kyiv(env=None, name="2", printlog = False)
        ibm_sherbrooke = IBM_Sherbrooke(env=None, name="3", printlog = False)
        ibm_quebec = IBM_Quebec(env=None, name="4", printlog = False)
        ibm_rensselaer = IBM_Rensselaer(env=None, name="5", printlog = False)
        ibm_brisbane = IBM_Brisbane(env=None, name="6", printlog = False)        
        ibm_brussels = IBM_Brussels(env=None, name="7", printlog = False)
        ibm_strasbourg = IBM_Strasbourg(env=None, name="8", printlog = False)        
        ibm_marrakesh = IBM_Marrakesh(env=None, name="9", printlog = False) 
        ibm_fez = IBM_Fez(env=None, name="10", printlog = False)
        ibm_torino = IBM_Torino(env=None, name="11", printlog = False)


        expovar_model = lambda: random.expovariate(lambd=LAMBDA)
        devices = [ibm_kawasaki, ibm_kyiv, ibm_sherbrooke, ibm_quebec, ibm_rensselaer, 
                  ibm_brisbane, ibm_brussels, ibm_strasbourg, ibm_marrakesh, ibm_fez, ibm_torino]

        start_time = time.time() # Start the timer

        # Initialize and run the simulation using jobs from a CSV file
        qcloudsimenv = QCloudSimEnv(
            devices=devices,
            broker_class=ParallelBroker,
            job_feed_method="generator",
            job_generation_model=expovar_model
        )
        qcloudsimenv.run(until=600)

        end_time = time.time() # End the timer
        elapsed_time = end_time - start_time # Calculate elapsed time

        print(f"Elapsed time: {elapsed_time:.5f} seconds\n")
        # Access job records after the simulation
        job_records = qcloudsimenv.job_records_manager.get_job_records()

        # Open a text file for writing
        with open(f"results/raw_records-lambda-{LAMBDA}.txt", "a") as file:
            file.write(f"Elapsed time: {elapsed_time:.5f} seconds\n")
            for job_id, events in job_records.items():
                # Write each job record to the file               
                file.write(f"Job ID: {job_id}, Events: {events}\n")
        processedjobs, notprocessedjobs = print_result(job_records, LAMBDA)
        processed[LAMBDA] = processedjobs
        notprocessed[LAMBDA] = notprocessedjobs

sim_end_time = time.time() # End the timer
sim_elapsed_time = sim_end_time - sim_start_time # Calculate elapsed time

print(f"Elapsed time: {sim_elapsed_time:.5f} seconds")

Elapsed time: 0.44549 seconds

LAMBDA: 0.5 | Total job processed: 246 | Never Assigned: 24
Finished: [11, 14, 11, 19, 21, 35, 26, 24, 32, 25, 28]
wait_time: [272.8351, 164.425, 215.8827, 169.7874, 167.4372, 35.6888, 24.8157, 33.6983, 35.1231, 32.9042, 26.6987]
Elapsed time: 0.65848 seconds

LAMBDA: 1 | Total job processed: 409 | Never Assigned: 118
Finished: [22, 20, 17, 21, 19, 51, 50, 54, 55, 58, 42]
wait_time: [200.2946, 174.2498, 205.6802, 207.1041, 205.9426, 34.7959, 28.6432, 30.7178, 31.1684, 29.9773, 33.5148]
Elapsed time: 1.06315 seconds

LAMBDA: 2 | Total job processed: 727 | Never Assigned: 392
Finished: [21, 16, 23, 26, 22, 102, 96, 87, 106, 132, 96]
wait_time: [211.5415, 244.8732, 185.9522, 165.0021, 170.5261, 37.4123, 30.2875, 27.868, 33.5739, 34.8886, 29.9857]
Elapsed time: 1.72842 seconds

LAMBDA: 3 | Total job processed: 963 | Never Assigned: 717
Finished: [25, 16, 18, 20, 18, 134, 155, 146, 146, 140, 145]
wait_time: [197.2104, 206.2391, 215.3324, 174.3659, 233.236, 38.